# Hvordan lage spørring som gir både kommunenavn og kommunenummer (id) fra JSON-stat

### Importerer biblioteker
Bruker [pyjstat](https://pypi.org/project/pyjstat/) og Pandas. Pandas importeres som del av pyjstat.

In [1]:
from pyjstat import pyjstat
import requests

URL med tabellens metadata, som vi skal poste spørringen mot

In [2]:
POST_URL = 'https://data.ssb.no/api/v0/no/table/07459'

Spørring - kan også tas fra konsoll.
0, 1 og 2-åringer i Oslo, Stavanger Trondheim i 2019 - 2025. (For kommuner berørt av regionreformen se lenger ned)

In [11]:
json_q = {"query":[{"code":"Region","selection":{"filter":"item","values":["0301","1103","5001"]}},
                   {"code":"Kjonn","selection":{"filter":"item","values":["1","2"]}},
                   {"code":"Alder","selection":{"filter":"item","values":["000","001","002"]}},
                   {"code":"ContentsCode","selection":{"filter":"item","values":["Personer1"]}},
                   {"code":"Tid","selection":{"filter":"item","values":["2019","2020","2021","2022","2023","2024","2025"]}}
                  ],
          "response":{"format":"json-stat2"}}

Poster spørringen

In [12]:
res = requests.post(POST_URL, json=json_q)

### Leser JSON-stat resultatet med biblioteket pyjstat
Lagrer det som datasett ds.

In [13]:
ds = pyjstat.Dataset.read(res.text)

In [14]:
type(ds)

pyjstat.pyjstat.Dataset

Se datasettet ds

In [15]:
# ds

### Henter noen overordnete metadata fra JSON-stat datasettet

In [16]:
tittel = ds['label']
print(tittel)

07459: Befolkning, etter region, kjønn, alder, statistikkvariabel og år


Sist Oppdatert som GMT

In [17]:
last_update = ds['updated']
print(last_update)

2025-02-25T07:00:00Z


Henter kilde

In [18]:
source = ds['source']
print(source)

Statistisk sentralbyrå


### Role gir noen snarveier, 
*men det er en liten bug i Statistikkbanken som ikke får med role - geo : 'Region' for akkurat denne tabellen, 07459*

In [19]:
ds_roles = ds['role']
print(ds_roles)

OrderedDict({'time': ['Tid'], 'geo': ['Region'], 'metric': ['ContentsCode']})


### Lager dataframes
Vi må lage to dataframes, en med tekst og en med koder/ID. Pyjstat tillater bare enten/eller

In [20]:
befolkning = ds.write('dataframe')
befolkning_id = ds.write('dataframe', naming='id')

Lager en ny kolonne med både kommunenummer og navn slått sammen. Kommunenummer er egentlig en streng på 4 tegn. 

In [21]:
befolkning['kommune'] = befolkning_id['Region'] + ' ' + befolkning['region']

Lager en ny dataframe som bare har med kolonnene vi ønsker, i ny rekkefølge. Merk doble [[ ]]

In [22]:
befolkning_ny = befolkning[['kommune', 'kjønn', 'alder', 'år', 'value']]
befolkning_ny.head(15)

,kommune,kjønn,alder,år,value
0,0301 Oslo,Kvinner,0 år,2019,4479
1,0301 Oslo,Kvinner,0 år,2020,4454
2,0301 Oslo,Kvinner,0 år,2021,4345
3,0301 Oslo,Kvinner,0 år,2022,4557
4,0301 Oslo,Kvinner,0 år,2023,3948
5,0301 Oslo,Kvinner,0 år,2024,4114
6,0301 Oslo,Kvinner,0 år,2025,4359
7,0301 Oslo,Kvinner,1 år,2019,4186
8,0301 Oslo,Kvinner,1 år,2020,4159
9,0301 Oslo,Kvinner,1 år,2021,4166


# Kommunereformen

Merk at spørringen i eksempelet over bruker rene kommunenummer, og ikke K-koder for sammenslåtte kommuner i tidsserier. 

En spørring som inkluderer tidsserier for kommuner som er berørt av kommunereformen, kan ikke benytte "item". K-koder må ha **agg:** og navnet på aggregeringen KommSummer istedet, "filter": "agg:KommSummer". Benytter også "agg:FemAarigGruppering" på alder.

Eksempel på en spørring for Oslo og Moss. Tallene for Oslo blir de samme som om jeg hadde brukt 0301, mens tidsserien for K-3103 Moss inkluderer kommunene som inngår i nye Moss, tidligere 0104 Moss og 0136 Rygge.

In [24]:
json_q1 = {"query":
           [{"code":"Region","selection":{"filter":"agg:KommSummer","values":["K-0301","K-3103"]}},
            {"code":"Kjonn","selection":{"filter":"item","values":["1","2"]}},
            {"code":"Alder","selection":{"filter":"agg:FemAarigGruppering", "values":["F00-04", "F05-09"]}},
            {"code":"ContentsCode","selection":{"filter":"item","values":["Personer1"]}},
            {"code":"Tid","selection":{"filter":"item","values":["2018","2019","2020","2021","2022","2023","2024","2025"]}}
           ],
           "response":{"format":"json-stat2"}}

## samme som over, men med ny spørring

In [25]:
res1 = requests.post(POST_URL, json=json_q1)
ds1 = pyjstat.Dataset.read(res1.text)
befolkning1 = ds1.write('dataframe')
befolkning1.head()

,region,kjønn,alder,statistikkvariabel,år,value
0,Moss,Kvinner,0-4 år,Personer,2018,1149
1,Moss,Kvinner,0-4 år,Personer,2019,1112
2,Moss,Kvinner,0-4 år,Personer,2020,1094
3,Moss,Kvinner,0-4 år,Personer,2021,1094
4,Moss,Kvinner,0-4 år,Personer,2022,1125


In [26]:
befolkning_id1 = ds1.write('dataframe', naming='id')
befolkning_id1.head()

,Region,Kjonn,Alder,ContentsCode,Tid,value
0,K-3103,2,F00-04,Personer1,2018,1149
1,K-3103,2,F00-04,Personer1,2019,1112
2,K-3103,2,F00-04,Personer1,2020,1094
3,K-3103,2,F00-04,Personer1,2021,1094
4,K-3103,2,F00-04,Personer1,2022,1125


(...)